In [ ]:
# Import the NetBird client
from netbird import APIClient
from netbird.exceptions import NetBirdAPIError, NetBirdNotFoundError
import os
from pprint import pprint
import json

print("✅ NetBird Python client imported successfully!")

# 🚀 NetBird Python Client Demo

This notebook demonstrates how to use the NetBird Python client to manage your NetBird network.

## 📋 What you'll learn:
- ✅ Connect to NetBird API
- ✅ Manage users, peers, and groups
- ✅ Configure policies and routes
- ✅ Monitor network activity
- ✅ Handle errors gracefully

In [ ]:
# Set up your NetBird credentials
# Option 1: Set environment variables (recommended)
NETBIRD_HOST = os.getenv("NETBIRD_HOST", "api.netbird.io")
NETBIRD_TOKEN = os.getenv("NETBIRD_API_TOKEN")

# Option 2: Set directly (for demo only - don't commit tokens!)
# NETBIRD_HOST = "your-netbird-server.com"
# NETBIRD_TOKEN = "your-api-token-here"

if not NETBIRD_TOKEN:
    print("⚠️  No API token found. Set NETBIRD_API_TOKEN environment variable.")
    print("   Example: export NETBIRD_API_TOKEN='your-token-here'")
else:
    print(f"✅ Using NetBird server: {NETBIRD_HOST}")
    print(f"✅ Token configured: {NETBIRD_TOKEN[:10]}...")

In [ ]:
import requests
import json

url = f"{NETBIRD_HOST}/api/users/current"

headers = {     
  'Accept': 'application/json',
  'Authorization': f'Token {NETBIRD_TOKEN}'
}

response = requests.request("GET", url, headers=headers)

(response.json())


## 🌐 Connect to NetBird

Create a client connection to your NetBird server.

In [ ]:
# Create NetBird client
client = APIClient(
    host=NETBIRD_HOST,
    api_token=NETBIRD_TOKEN,
    timeout=30.0
)

print(f"🔗 Connected to: {client.base_url}")
print(f"🔐 Authentication: Token configured")
print(f"⏱️  Timeout: {client.timeout} seconds")

## 👤 User Management

Let's start by exploring users in your NetBird network.

In [ ]:
# Get current user information
try:
    current_user = client.users.get_current()
    print("👤 Current User:")
    print(f"   Name: {current_user['name']}")
    print(f"   Email: {current_user['email']}")
    print(f"   Role: {current_user['role']}")
    print(f"   Status: {current_user['status']}")
    print(f"   Last Login: {current_user['last_login']}")
except NetBirdAPIError as e:
    print(f"❌ Error getting current user: {e}")

In [ ]:
# List all users
try:
    users = client.users.list()
    print(f"👥 Found {len(users)} users:")
    print()
    
    for user in users:
        print(f"   📧 {user['email'] or 'No email'}")
        print(f"      Role: {user['role']} | Status: {user['status']}")
        print(f"      Service User: {user['is_service_user']} | Blocked: {user['is_blocked']}")
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing users: {e}")

## 🖥️ Peer Management

Explore the devices (peers) connected to your NetBird network.

In [ ]:
# List all peers
try:
    peers = client.peers.list()
    print(f"🖥️  Found {len(peers)} peers:")
    print()
    
    for peer in peers:
        status = "🟢 Online" if peer['connected'] else "🔴 Offline"
        print(f"   {status} {peer['name']}")
        print(f"      IP: {peer['ip']} | OS: {peer['os']}")
        print(f"      Version: {peer['version']} | Last Seen: {peer['last_seen']}")
        
        # Show geographic info if available
        if peer.get('city_name') and peer.get('country_code'):
            print(f"      Location: {peer['city_name']}, {peer['country_code']}")
        
        # Show groups
        if peer.get('groups'):
            group_names = [g.get('name', 'Unknown') for g in peer['groups']]
            print(f"      Groups: {', '.join(group_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing peers: {e}")

## 👥 Group Management

Manage groups to organize your network peers.

In [ ]:
# List all groups
try:
    groups = client.groups.list()
    print(f"👥 Found {len(groups)} groups:")
    print()
    
    for group in groups:
        print(f"   📁 {group['name']}")
        print(f"      ID: {group['id']}")
        print(f"      Peers: {group['peers_count']}")
        print(f"      Resources: {group.get('resources_count', 0)}")
        
        # Show peer details if available
        if group.get('peers') and len(group['peers']) <= 5:  # Don't show too many
            peer_names = [p.get('name', 'Unknown') for p in group['peers']]
            print(f"      Members: {', '.join(peer_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing groups: {e}")

## 🔐 Policy Management

View and manage network access policies.

In [ ]:
# List all policies
try:
    policies = client.policies.list()
    print(f"🔐 Found {len(policies)} policies:")
    print()
    
    for policy in policies:
        status = "✅ Enabled" if policy['enabled'] else "❌ Disabled"
        print(f"   {status} {policy['name']}")
        print(f"      Description: {policy.get('description') or 'No description'}")
        print(f"      Rules: {len(policy.get('rules', []))}")
        
        # Show rule details
        if policy.get('rules'):
            for rule in policy['rules'][:2]:  # Show first 2 rules
                action = "🟢 Allow" if rule.get('action') == 'accept' else "🔴 Deny"
                print(f"        {action} {rule.get('name', 'Unnamed rule')}")
                print(f"          Protocol: {rule.get('protocol', 'any')}")
                
                # Show sources and destinations
                sources = rule.get('sources', [])
                destinations = rule.get('destinations', [])
                if sources:
                    src_names = [s.get('name', 'Unknown') for s in sources[:2]]
                    print(f"          From: {', '.join(src_names)}")
                if destinations:
                    dst_names = [d.get('name', 'Unknown') for d in destinations[:2]]
                    print(f"          To: {', '.join(dst_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing policies: {e}")

## 🌐 Network Information

View network configurations and routes.

In [ ]:
# List networks
try:
    networks = client.networks.list()
    print(f"🌐 Found {len(networks)} networks:")
    print()
    
    for network in networks:
        print(f"   📡 {network['name']}")
        print(f"      Description: {network.get('description') or 'No description'}")
        print(f"      Routers: {len(network.get('routers', []))}")
        print(f"      Resources: {len(network.get('resources', []))}")
        print(f"      Policies: {len(network.get('policies', []))}")
        print(f"      Routing Peers: {network.get('routing_peers_count', 0)}")
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing networks: {e}")

In [ ]:
# List routes
try:
    routes = client.routes.list()
    print(f"🛣️  Found {len(routes)} routes:")
    
    if routes:
        print()
        for route in routes:
            status = "✅ Enabled" if route['enabled'] else "❌ Disabled"
            print(f"   {status} {route.get('description') or 'Unnamed route'}")
            print(f"      Network: {route['network_id']}")
            print(f"      Metric: {route['metric']}")
            print()
    else:
        print("   No routes configured.")
        
except NetBirdAPIError as e:
    print(f"❌ Error listing routes: {e}")

## 🔑 Setup Keys

Manage setup keys for adding new devices.

In [ ]:
# List setup keys
try:
    setup_keys = client.setup_keys.list()
    print(f"🔑 Found {len(setup_keys)} setup keys:")
    print()
    
    for key in setup_keys:
        status = "✅ Valid" if key['valid'] and not key['revoked'] else "❌ Invalid"
        print(f"   {status} {key['name']}")
        print(f"      Type: {key['type']}")
        print(f"      Used: {key['used_times']} times")
        print(f"      Limit: {key.get('usage_limit') or 'Unlimited'}")
        print(f"      Expires: {key.get('expires') or 'Never'}")
        print(f"      Last Used: {key.get('last_used') or 'Never'}")
        
        if key.get('auto_groups'):
            print(f"      Auto Groups: {len(key['auto_groups'])} groups")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing setup keys: {e}")

## 🏢 Account Information

View account settings and configuration.

In [ ]:
# List accounts
try:
    accounts = client.accounts.list()
    print(f"🏢 Found {len(accounts)} accounts:")
    print()
    
    for account in accounts:
        print(f"   🏷️  {account['domain']}")
        print(f"      Category: {account['domain_category']}")
        print(f"      Created: {account['created_at']}")
        print(f"      Created By: {account['created_by']}")
        
        # Show settings if available
        if account.get('settings'):
            settings = account['settings']
            print(f"      Settings:")
            
            # Show key settings
            if isinstance(settings, dict):
                if 'peer_login_expiration_enabled' in settings:
                    print(f"        Login Expiration: {settings['peer_login_expiration_enabled']}")
                if 'groups_propagation_enabled' in settings:
                    print(f"        Group Propagation: {settings['groups_propagation_enabled']}")
                if 'lazy_connection_enabled' in settings:
                    print(f"        Lazy Connection: {settings['lazy_connection_enabled']}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing accounts: {e}")

## 📊 Activity & Events

Monitor network activity and audit events.

In [ ]:
# Get recent audit events
try:
    events = client.events.get_audit_events()
    print(f"📊 Found {len(events)} recent events:")
    print()
    
    # Show last 5 events
    for event in events[:5]:
        print(f"   📅 {event['timestamp']}")
        print(f"      Activity: {event['activity']}")
        
        if event.get('initiator_name'):
            print(f"      Initiator: {event['initiator_name']}")
        if event.get('initiator_email'):
            print(f"      Email: {event['initiator_email']}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error getting events: {e}")

## 🔧 DNS Management

Configure DNS settings for your network.

In [ ]:
# Get DNS settings
try:
    dns_settings = client.dns.get_settings()
    print("🔧 DNS Settings:")
    print(f"   Disabled Management Groups: {len(dns_settings.get('disabled_management_groups', []))}")
    print()
    
except NetBirdAPIError as e:
    print(f"❌ Error getting DNS settings: {e}")
    print()

In [ ]:
# List DNS nameserver groups
try:
    nameserver_groups = client.dns.list_nameserver_groups()
    print(f"🌐 Found {len(nameserver_groups)} DNS nameserver groups:")
    
    if nameserver_groups:
        print()
        for group in nameserver_groups:
            status = "✅ Enabled" if group['enabled'] else "❌ Disabled"
            print(f"   {status} {group['name']}")
            print(f"      Nameservers: {', '.join(group['nameservers'])}")
            if group.get('description'):
                print(f"      Description: {group['description']}")
            print()
    else:
        print("   No DNS nameserver groups configured.")
        
except NetBirdAPIError as e:
    print(f"❌ Error listing DNS nameserver groups: {e}")

## 🎯 Create/Update Operations

Demonstrate how to create and update resources (use with caution!).

In [ ]:
# Example: Create a new group (uncomment to run)
from netbird.models import GroupCreate

# UNCOMMENT THE LINES BELOW TO CREATE A TEST GROUP:
try:
    # Create a test group
    group_data = GroupCreate(
        name="Demo-Group-from-Jupyter",
        peers=[]  # Start with no peers
    )
    
    new_group = client.groups.create(group_data)
    print(f"✅ Created group: {new_group['name']} (ID: {new_group['id']})")
    
    # Clean up - delete the group immediately
    client.groups.delete(new_group['id'])
    print(f"🗑️  Deleted test group: {new_group['name']}")
    
except NetBirdAPIError as e:
    print(f"❌ Error with group operations: {e}")

print("💡 Uncomment the code above to test group creation/deletion")

## 🛡️ Error Handling Examples

Demonstrate proper error handling patterns.

In [ ]:
# Example: Handling different types of errors
from netbird.exceptions import (
    NetBirdAPIError,
    NetBirdNotFoundError,
    NetBirdAuthenticationError,
    NetBirdValidationError
)

def safe_api_call(operation_name, api_function, *args, **kwargs):
    """Safely execute an API call with proper error handling."""
    try:
        result = api_function(*args, **kwargs)
        print(f"✅ {operation_name}: Success")
        return result
    
    except NetBirdAuthenticationError as e:
        print(f"🔐 {operation_name}: Authentication failed - check your API token")
        return None
    
    except NetBirdNotFoundError as e:
        print(f"🔍 {operation_name}: Resource not found")
        return None
    
    except NetBirdValidationError as e:
        print(f"📝 {operation_name}: Validation error - {e}")
        return None
    
    except NetBirdAPIError as e:
        print(f"⚠️  {operation_name}: API error (status {e.status_code}) - {e}")
        return None
    
    except Exception as e:
        print(f"💥 {operation_name}: Unexpected error - {e}")
        return None

# Test error handling
print("🧪 Testing error handling:")

# This should fail - invalid group ID
result = safe_api_call(
    "Get invalid group",
    client.groups.get,
    "invalid-group-id-12345"
)

# This should succeed - list users
result = safe_api_call(
    "List users",
    client.users.list
)

print("\n💡 Use this pattern in your production code!")

## 📈 Network Statistics Summary

Get a complete overview of your NetBird network.

In [ ]:
# Generate network summary
print("📈 NetBird Network Summary")
print("=" * 50)

summary = {}

# Collect statistics
try:
    users = client.users.list()
    summary['total_users'] = len(users)
    summary['admin_users'] = len([u for u in users if u['role'] == 'admin'])
    summary['active_users'] = len([u for u in users if u['status'] == 'active'])
except:
    summary['total_users'] = 'Error'

try:
    peers = client.peers.list()
    summary['total_peers'] = len(peers)
    summary['online_peers'] = len([p for p in peers if p['connected']])
    summary['offline_peers'] = len([p for p in peers if not p['connected']])
except:
    summary['total_peers'] = 'Error'

try:
    groups = client.groups.list()
    summary['total_groups'] = len(groups)
except:
    summary['total_groups'] = 'Error'

try:
    policies = client.policies.list()
    summary['total_policies'] = len(policies)
    summary['enabled_policies'] = len([p for p in policies if p['enabled']])
except:
    summary['total_policies'] = 'Error'

try:
    routes = client.routes.list()
    summary['total_routes'] = len(routes)
except:
    summary['total_routes'] = 'Error'

try:
    setup_keys = client.setup_keys.list()
    summary['total_setup_keys'] = len(setup_keys)
    summary['valid_setup_keys'] = len([k for k in setup_keys if k['valid'] and not k['revoked']])
except:
    summary['total_setup_keys'] = 'Error'

# Display summary
print(f"👥 Users:          {summary.get('total_users', 'N/A')} total, {summary.get('admin_users', 'N/A')} admins, {summary.get('active_users', 'N/A')} active")
print(f"🖥️  Peers:          {summary.get('total_peers', 'N/A')} total, {summary.get('online_peers', 'N/A')} online, {summary.get('offline_peers', 'N/A')} offline")
print(f"👥 Groups:         {summary.get('total_groups', 'N/A')}")
print(f"🔐 Policies:       {summary.get('total_policies', 'N/A')} total, {summary.get('enabled_policies', 'N/A')} enabled")
print(f"🛣️  Routes:         {summary.get('total_routes', 'N/A')}")
print(f"🔑 Setup Keys:     {summary.get('total_setup_keys', 'N/A')} total, {summary.get('valid_setup_keys', 'N/A')} valid")

print("\n✅ NetBird Python Client Demo Complete!")

## 📊 Network Diagram Generation

The NetBird client includes powerful network topology visualization capabilities! Generate diagrams in multiple formats to visualize your network architecture, relationships, policies, and resource access patterns.

### Available Formats:
- **Mermaid**: Interactive web-based diagrams (great for documentation)
- **Graphviz**: Professional network diagrams with advanced layouts
- **Python Diagrams**: Cloud architecture style diagrams

### Features:
- 🎨 Dynamic color coding for different source groups
- 🔐 Policy visualization with access rules
- 🌐 Resource and router mapping
- 📁 Group-based organization
- 💾 Export to multiple file formats

In [ ]:
# Generate a Mermaid diagram (default format)
try:
    print("📊 Generating Mermaid network diagram...")
    
    # Generate mermaid diagram without saving to file
    mermaid_content = client.generate_diagram(format="mermaid")
    
    if mermaid_content:
        print("✅ Mermaid diagram generated successfully!")
        print(f"📝 Diagram length: {len(mermaid_content)} characters")
        
        # Display the Mermaid diagram inline using IPython display
        from IPython.display import display, Markdown
        
        # Create markdown with mermaid code block for rendering
        mermaid_markdown = f"""
```mermaid
{mermaid_content}
```
        """
        
        print("\n🎨 Rendering Mermaid diagram:")
        display(Markdown(mermaid_markdown))
        
        print("\n💡 Tips:")
        print("   - The diagram should render above if your Jupyter environment supports Mermaid")
        print("   - Copy the code to https://mermaid.live/ for interactive viewing")
        print("   - Use in GitHub README files for automatic rendering")
        print("   - Great for documentation and presentations")
            
    else:
        print("❌ No diagram generated - check if you have networks configured")
        
except Exception as e:
    print(f"❌ Error generating diagram: {e}")

In [ ]:
# Generate all diagram formats for comparison
try:
    print("🎨 Generating all diagram formats for comparison...")
    print("=" * 60)
    
    # 1. Generate Mermaid
    print("\n1️⃣ Mermaid Diagram:")
    mermaid_content = client.generate_diagram(format="mermaid")
    if mermaid_content:
        print(f"✅ Generated ({len(mermaid_content)} chars)")
        from IPython.display import display, Markdown
        display(Markdown(f"```mermaid\n{mermaid_content}\n```"))
    
    # 2. Generate Graphviz  
    print("\n2️⃣ Graphviz Diagram:")
    graphviz_content = client.generate_diagram(format="graphviz")
    if graphviz_content:
        print(f"✅ Generated ({len(graphviz_content)} chars)")
        try:
            import graphviz
            graph = graphviz.Source(graphviz_content)
            display(graph)
        except ImportError:
            print("   📝 Graphviz library not available for rendering")
            display(Markdown(f"```dot\n{graphviz_content[:500]}...\n```"))
    
    # 3. Generate Python Diagrams
    print("\n3️⃣ Python Diagrams:")
    import tempfile
    import os
    
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_file = os.path.join(temp_dir, "comparison_diagram")
        result = client.generate_diagram(format="diagrams", output_file=temp_file)
        
        if result is None:
            png_file = f"{temp_file}.png"
            if os.path.exists(png_file):
                print("✅ Generated PNG diagram")
                from IPython.display import Image
                display(Image(png_file))
            else:
                print("❌ PNG file not found")
        else:
            print("❌ Generation failed")
    
    print("\n💡 All diagram formats rendered above for comparison!")
    
except Exception as e:
    print(f"❌ Error generating comparison diagrams: {e}")

# 1. 🧩 Mermaid Diagram Example
print("🧩 MERMAID DIAGRAM")
print("=" * 50)
print("Best for: Documentation, GitHub README, Interactive web viewing")
print()

try:
    mermaid_diagram = client.generate_diagram(
        format="mermaid",
        include_routers=True,
        include_policies=True,
        include_resources=True
    )
    
    if mermaid_diagram:
        print("✅ Mermaid diagram generated successfully!")
        
        # Show structure info
        lines = mermaid_diagram.split('\n')
        print(f"📊 Total lines: {len(lines)}")
        
        # Display inline
        from IPython.display import display, Markdown
        
        mermaid_markdown = f"""
**🎨 Interactive Mermaid Diagram:**

```mermaid
{mermaid_diagram}
```
        """
        
        display(Markdown(mermaid_markdown))
        
        print("\n🎯 Mermaid Features:")
        print("   - Interactive nodes (clickable)")
        print("   - Color-coded by source groups") 
        print("   - Policy labels on connections")
        print("   - Supports GitHub rendering")
        print("   - Web-based live editor support")
        
    else:
        print("❌ No mermaid diagram generated")
        
except Exception as e:
    print(f"❌ Error generating mermaid diagram: {e}")

In [ ]:
# 2. 🔗 Graphviz Diagram Example  
print("🔗 GRAPHVIZ DIAGRAM")
print("=" * 50)
print("Best for: Professional presentations, Network analysis, Print media")
print()

try:
    graphviz_diagram = client.generate_diagram(
        format="graphviz",
        include_routers=True,
        include_policies=True,
        include_resources=True
    )
    
    if graphviz_diagram:
        print("✅ Graphviz diagram generated successfully!")
        
        # Show structure analysis
        lines = graphviz_diagram.split('\n')
        print(f"📊 Total lines: {len(lines)}")
        
        # Count different elements
        node_count = len([line for line in lines if ' [' in line and 'label=' in line])
        edge_count = len([line for line in lines if ' -> ' in line])
        
        print(f"📍 Nodes: {node_count}")
        print(f"🔗 Edges: {edge_count}")
        
        # Try to render with graphviz if available
        try:
            import graphviz
            from IPython.display import display
            
            # Create graphviz object and display
            graph = graphviz.Source(graphviz_diagram)
            print("\n🎨 Rendered Graphviz Diagram:")
            display(graph)
            
        except ImportError:
            # Fallback to showing code
            from IPython.display import display, Markdown
            
            graphviz_markdown = f"""
**🔗 Graphviz Diagram Code:**

```dot
{graphviz_diagram}
```

*Install graphviz for inline rendering: `pip install graphviz`*
            """
            display(Markdown(graphviz_markdown))
        
        print("\n🎯 Graphviz Features:")
        print("   - Professional layout algorithms")
        print("   - High-quality vector output")
        print("   - Advanced styling options")
        print("   - Multiple output formats (SVG, PNG, PDF)")
        print("   - Hierarchical node positioning")
        
    else:
        print("❌ No graphviz diagram generated")
        
except Exception as e:
    print(f"❌ Error generating graphviz diagram: {e}")

In [ ]:
# 3. 🏗️ Python Diagrams Example
print("🏗️ PYTHON DIAGRAMS")
print("=" * 50)
print("Best for: Cloud architecture, Infrastructure diagrams, Visual presentations")
print()

try:
    # For inline rendering, we need to create a temporary diagram
    import tempfile
    import os
    
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_file = os.path.join(temp_dir, "temp_diagram")
        
        # Python diagrams format returns None on success and creates PNG files
        result = client.generate_diagram(
            format="diagrams",
            output_file=temp_file,
            include_routers=True,
            include_policies=True,
            include_resources=True
        )
        
        # For diagrams format, None means success
        if result is None:
            print("✅ Python Diagrams generated successfully!")
            
            png_file = f"{temp_file}.png"
            if os.path.exists(png_file):
                file_size = os.path.getsize(png_file)
                print(f"📊 File size: {file_size:,} bytes")
                
                # Display the image inline
                from IPython.display import display, Image, Markdown
                
                print("\n🎨 Rendered Python Diagram:")
                display(Image(png_file))
                
                print(f"✅ Diagram rendered inline above")
                
            else:
                print("⚠️  PNG file not found in temp directory")
            
            print("\n🎯 Python Diagrams Features:")
            print("   - Generic network icons (routers, internet nodes)")
            print("   - Professional infrastructure styling")
            print("   - Automatic clustering and grouping")
            print("   - High-resolution PNG output")
            print("   - Perfect for presentations and documentation")
            
        else:
            print("❌ Python Diagrams generation failed")
            print(f"   Result: {result}")
        
except Exception as e:
    print(f"❌ Error generating Python diagrams: {e}")
    print("   Note: Requires 'diagrams' library: pip install diagrams")

In [ ]:
### 📊 Format Comparison

| Format | Best For | Output | Interactive | Inline Rendering | Complexity |
|--------|----------|--------|-------------|------------------|------------|
| **Mermaid** | Documentation, GitHub | Text/SVG | ✅ Yes | ✅ Markdown | Simple |
| **Graphviz** | Professional diagrams | SVG/PNG/PDF | ❌ No | ✅ Python lib | Advanced |
| **Python Diagrams** | Architecture presentations | PNG | ❌ No | ✅ Image display | Simple |

### 🎯 When to Use Each Format:

- **📝 Use Mermaid when:**
  - Adding diagrams to README files
  - Creating interactive documentation
  - Working with GitHub/GitLab wikis
  - Need lightweight, text-based diagrams
  - Want inline rendering in Jupyter notebooks

- **🎨 Use Graphviz when:**
  - Creating professional network analysis
  - Need precise layout control
  - Generating high-quality print materials
  - Complex network visualization requirements
  - Want vector-based output

- **🏗️ Use Python Diagrams when:**
  - Presenting to stakeholders
  - Creating architecture documentation
  - Need visually appealing cloud-style diagrams
  - Building presentation materials
  - Want PNG images for easy sharing

### 🔧 Jupyter Notebook Rendering:

All diagram formats can be rendered inline in Jupyter notebooks:
- **Mermaid**: Rendered as Markdown code blocks (requires Mermaid support)
- **Graphviz**: Rendered directly using `graphviz` Python library
- **Python Diagrams**: Displayed as inline PNG images

No files are saved to disk - everything renders directly in the notebook!

# Customize diagram generation with options
try:
    print("🎨 Generating customized network diagram...")
    
    # Generate diagram with specific options
    custom_diagram = client.generate_diagram(
        format="mermaid",
        include_routers=False,      # Exclude routers for simpler view
        include_policies=True,      # Show policies
        include_resources=True      # Show resources
    )
    
    if custom_diagram:
        print("✅ Customized diagram generated!")
        print("🔧 Configuration:")
        print("   - Format: Mermaid")
        print("   - Routers: Hidden")
        print("   - Policies: Shown")  
        print("   - Resources: Shown")
        print(f"   - Length: {len(custom_diagram)} characters")
        
        # Display the customized diagram
        from IPython.display import display, Markdown
        
        custom_markdown = f"""
**🎯 Customized Diagram (No Routers):**

```mermaid
{custom_diagram}
```
        """
        
        display(Markdown(custom_markdown))
    
    print("\n🎯 Available Options:")
    print("   - format: 'mermaid', 'graphviz', 'diagrams'")
    print("   - include_routers: Show/hide network routers")
    print("   - include_policies: Show/hide access policies")
    print("   - include_resources: Show/hide network resources")
    print("\n💡 Compare this simplified view with the full diagram above!")
        
except Exception as e:
    print(f"❌ Error generating custom diagram: {e}")

In [ ]:
# Customize diagram generation with options
try:
    print("🎨 Generating customized network diagram...")
    
    # Generate diagram with specific options
    custom_diagram = client.generate_diagram(
        format="mermaid",
        include_routers=False,      # Exclude routers for simpler view
        include_policies=True,      # Show policies
        include_resources=True      # Show resources
    )
    
    if custom_diagram:
        print("✅ Customized diagram generated!")
        print("🔧 Configuration:")
        print("   - Format: Mermaid")
        print("   - Routers: Hidden")
        print("   - Policies: Shown")  
        print("   - Resources: Shown")
        print(f"   - Length: {len(custom_diagram)} characters")
        
        # Show a sample of the content
        sample_lines = custom_diagram.split('\n')[:5]
        print("\n📋 Sample content:")
        for line in sample_lines:
            print(f"   {line}")
    
    print("\n🎯 Available Options:")
    print("   - format: 'mermaid', 'graphviz', 'diagrams'")
    print("   - output_file: Save to files (optional)")
    print("   - include_routers: Show/hide network routers")
    print("   - include_policies: Show/hide access policies")
    print("   - include_resources: Show/hide network resources")
        
except Exception as e:
    print(f"❌ Error generating custom diagram: {e}")

In [ ]:
### 🎯 Diagram Use Cases

The network diagram generation feature is perfect for:

1. **📖 Documentation**
   - Include diagrams in README files
   - Generate architecture documentation
   - Create onboarding materials for new team members
   - **✨ Jupyter notebooks with inline rendering**

2. **🔍 Network Analysis**
   - Visualize complex policy relationships
   - Identify potential security gaps
   - Understand resource access patterns
   - **✨ Interactive exploration in notebooks**

3. **🎨 Presentations**
   - Export professional diagrams for meetings
   - Show network architecture to stakeholders
   - Demonstrate network growth over time
   - **✨ Live demos in Jupyter presentations**

4. **🔧 Troubleshooting**
   - Quickly identify connectivity issues
   - Visualize network topology for debugging
   - Understand traffic flow patterns
   - **✨ Real-time network state visualization**

5. **📊 Monitoring & Reporting**
   - Generate automated network reports
   - Track network changes over time
   - Create compliance documentation
   - **✨ Interactive dashboards and analysis**

### 🚀 Jupyter Notebook Benefits:

- **No file management**: Diagrams render directly inline
- **Interactive exploration**: Modify parameters and see results immediately  
- **Version control friendly**: Notebook contains everything needed
- **Shareable**: Send notebooks with embedded diagrams
- **Live updates**: Regenerate diagrams as your network changes

---

## 🎉 Conclusion

You've now explored the complete NetBird Python client capabilities, from basic network management to advanced diagram generation with inline Jupyter rendering! This powerful combination gives you everything needed to manage, monitor, and visualize your NetBird networks programmatically.

**🚀 Happy networking with NetBird!**

### 🎯 Diagram Use Cases

The network diagram generation feature is perfect for:

1. **📖 Documentation**
   - Include diagrams in README files
   - Generate architecture documentation
   - Create onboarding materials for new team members

2. **🔍 Network Analysis**
   - Visualize complex policy relationships
   - Identify potential security gaps
   - Understand resource access patterns

3. **🎨 Presentations**
   - Export professional diagrams for meetings
   - Show network architecture to stakeholders
   - Demonstrate network growth over time

4. **🔧 Troubleshooting**
   - Quickly identify connectivity issues
   - Visualize network topology for debugging
   - Understand traffic flow patterns

5. **📊 Monitoring & Reporting**
   - Generate automated network reports
   - Track network changes over time
   - Create compliance documentation

---

## 🎉 Conclusion

You've now explored the complete NetBird Python client capabilities, from basic network management to advanced diagram generation! This powerful combination gives you everything needed to manage, monitor, and visualize your NetBird networks programmatically.

**🚀 Happy networking with NetBird!**

## 🎓 Next Steps

Now that you've seen the NetBird Python client in action, here are some ideas for your own projects:

### 🔧 Automation Scripts
- Automatically add new devices to specific groups
- Monitor peer connectivity and send alerts
- Backup and restore network configurations

### 📊 Monitoring & Analytics
- Generate network usage reports
- Track peer connection patterns
- Monitor policy effectiveness

### 🌐 Integration Projects
- Integrate with infrastructure automation tools
- Build custom dashboards
- Create API webhooks for real-time updates

### 📚 Resources
- **Documentation**: Check the `/tests/` folder for more examples
- **API Reference**: NetBird API documentation
- **GitHub**: Contribute to the project or report issues

---

**Happy networking with NetBird! 🚀**